<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="4" color="black"><b>From Keras Experiment to deployment with watson-machine-learning-client and Hyperparameter Optimization</b></font></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr> 
   <tr style="border: none">
       <td style="border: none"><img src="https://github.com/pmservice/wml-sample-models/raw/master/tensorflow/hand-written-digit-recognition/images/experiment_banner.png" width="600" height = "200" alt="Icon"></td>
   </tr>
</table>

This notebook contains steps and code to demonstrate support of deep learning experiments in Watson Machine Learning Service. This notebook introduces commands for getting data, training_definition persistance, experiment training, model persistance, model deployment and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.


## Learning goals

The learning goals of this notebook are:

-  Working with Watson Machine Learning experiments to train Deep Learning models (Keras).
-  Saving trained models in Watson Machine Learning repository.
-  Online deployment and scoring of trained model.


## Contents

This notebook contains the following parts:

1.	[Set up the environment](#setup)
2.	[Training definition](#definition)
3.	[Experiment definition](#experiment)
4.	[Experiment Run](#run)
5.	[Deploy](#deploy)
6. [Scoring](#scoring)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning (WML) Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered and information about how to create the instance is [here](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html))
-  Create a [Cloud Object Storage (COS)](https://console.bluemix.net/catalog/infrastructure/cloud-object-storage) instance (a lite plan is offered and information about how to order storage is [here](https://console.bluemix.net/docs/services/cloud-object-storage/basics/order-storage.html#order-storage)).
    - After you create COS instance, go to your COS dashboard.
    - In **Service credentials** tab, click **New Credential**.
    - Add the inline configuration parameter: {["HMAC"](https://console.bluemix.net/docs/services/cloud-object-storage/hmac/credentials.html#using-hmac-credentials):true}, click **Add**.

    This configuration parameter adds the following section to the instance credentials, (for use later in this notebook):
    ```
      "cos_hmac_keys": {
            "access_key_id": "722432c254bc4eaa96e05897bf2779e2",
            "secret_access_key": "286965ac10ecd4de8b44306288c7f5a3e3cf81976a03075c"
       }
       ```

### 1.1 Work with Cloud Object Storage (COS)

Import the Boto library, which allows Python developers to manage COS.

**Tip:** If `ibm_boto3` is not preinstalled in you environment, run the following command to install it: 

In [ ]:
# Run command if ibm_boto3 is not installed.
#!pip install ibm-cos-sdk

In [ ]:
from project_lib import Project
project = Project(project_id='4869a883-0dd1-4768-88d9-c789a68ff5f0', project_access_token='p-855bacd87bd3bbe331c93b14dd2e79fe6d23c918')
pc = project.project_context

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config
import os
import json
import warnings
import time

Define the endpoint you will use. 

1. To do this, first enter your COS credentials in the following cell. 
You can find these credentials in your COS instance dashboard under the **Service credentials** tab.
**Note** the HMAC key, described in [set up the environment](#setup) is included in these credentials.

2. Go to the **Endpoint** tab in the COS instance's dashboard to get the endpoint information.

In [ ]:
# your COS credentials
cos_credentials = {
  "apikey": "***",
  "cos_hmac_keys": {
    "access_key_id": "***",
    "secret_access_key": "***"
  },
  "endpoints": "https://cos-service.bluemix.net/endpoints",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:cloud-object-storage:global:a/7d7bba8d3af690913ac4403733b01605:5cbade09-286a-47de-ab17-7fc51ba1a373::",
  "iam_apikey_name": "auto-generated-apikey-dd670e5e-3668-4fb3-804e-b21fe014b81e",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
  "iam_serviceid_crn": "***",
  "resource_instance_id": "***"
}



api_key = cos_credentials['apikey']
service_instance_id = cos_credentials['resource_instance_id']
auth_endpoint = 'https://iam.bluemix.net/oidc/token'
service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'

# our bucket names
buckets = ['fashion-mnist-training-data-massachi-1', 'fashion-mnist-results-data-massachi-1']


We create Boto resource by providing type, endpoint_url and credentials.

In [ ]:
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=api_key,
                         ibm_service_instance_id=service_instance_id,
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

### 1.2 Work with the Watson Machine Learning instance

Authenticate to Watson Machine Learning service on IBM Cloud.

In [ ]:
import urllib3, requests, json, base64, time, os
warnings.filterwarnings('ignore')

Authenticate to the Watson Machine Learning service on IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the [Service Credentials](https://console.bluemix.net/docs/services/service_credentials.html#service_credentials) tab of the service instance that you created on IBM Cloud. <BR>If you cannot see the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your Watson Machine Learning service instance credentials here.

In [ ]:
wml_credentials = {
  "url": "https://ibm-watson-ml.mybluemix.net",
  "username": "***",
  "password": "***",
  "instance_id": "***"
}

#### Install `watson-machine-learning-client` from pypi

In [ ]:
!pip install --upgrade --quiet watson-machine-learning-client

#### Import `watson-machine-learning-client` and authenticate to service instance

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
client.training.list()

<a id="definition"></a>
## 2. Training definitions

### Save training definition 

#### Prepare training definition metadata

In [ ]:
model_definition_1_metadata = {
            client.repository.DefinitionMetaNames.NAME: "DLDemos-HPO-Notebook",
            client.repository.DefinitionMetaNames.FRAMEWORK_NAME: "tensorflow",
            client.repository.DefinitionMetaNames.FRAMEWORK_VERSION: "1.5",
            client.repository.DefinitionMetaNames.RUNTIME_NAME: "python",
            client.repository.DefinitionMetaNames.RUNTIME_VERSION: "3.5",
            client.repository.DefinitionMetaNames.EXECUTION_COMMAND: "python3 training_hpo.py"
            }

In [ ]:
# get the experiment zip and write it to local
with open('experiment.zip', 'wb') as f:
    f.write(project.get_file('hpo-training-def.zip').read())

#### Publish training definition in Watson Machine Learning repository

##### Store definition 1

In [ ]:
definition_1_details = client.repository.store_definition('experiment.zip', model_definition_1_metadata)

definition_1_url = client.repository.get_definition_url(definition_1_details)
definition_1_uid = client.repository.get_definition_uid(definition_1_details)
print(definition_1_url)

#### LIST stored definitions

In [ ]:
client.repository.list_definitions()

<a id="experiment"></a>
## 3. Experiment definition

### Save experiment

#### Get the list of supported configuration params

In [ ]:
client.repository.ExperimentMetaNames.show()

#### Experiment configuration dictionary
Create experiment that will train models based on previously stored definitions.

`TRAINING_DATA_REFERENCE` - location of traininng data

In [ ]:
TRAINING_DATA_REFERENCE = {
                            "connection": {
                                "endpoint_url": service_endpoint,
                                "aws_access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
                                "aws_secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
                            },
                            "source": {
                                "bucket": buckets[0],
                            },
                            "type": "s3"
}

`TRAINING_RESULTS_REFERENCE` - location of training results

In [ ]:
TRAINING_RESULTS_REFERENCE = {
                                "connection": {
                                    "endpoint_url": service_endpoint,
                                    "aws_access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
                                    "aws_secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
                                },
                                "target": {
                                    "bucket": buckets[1],
                                },
                                "type": "s3"
}

Configure hyper parameters optimizer for you experiment. We are interested in `val_acc` metric so we need to put it as optimizer `objective`. `num_optimizer_steps` tells the optimizer how many models we want to train based on hyper parameters values combinations.

In [ ]:
HPO = {
        "method": {
            "name": "rbfopt", # name of the algo -- choose rbfopt
            "parameters": [
                client.experiments.HPOMethodParam("objective", "accuracy"),
                client.experiments.HPOMethodParam("maximize_or_minimize", "maximize"),
                client.experiments.HPOMethodParam("num_optimizer_steps", 10)
            ]
        },
        "hyper_parameters": [
            client.experiments.HPOParameter('learning_rate', min=0.0001, max=0.01, step=0.0005),
            client.experiments.HPOParameter('dropout', min=0.01, max=0.99, step=0.1),
            client.experiments.HPOParameter('conv_filter_1', min=64, max=256, step=32),
            client.experiments.HPOParameter('batch_size', min=32, max=256, step=32),
            
            
        ]
     }          

Configure your experiment. `TRAINING_REFERENCES` links previously stored training definitions and provides information about `compute_configuration` that will be used to run the training.

In [ ]:
experiment_metadata = {
            client.repository.ExperimentMetaNames.NAME: "FASHION-MNIST-HPO-May-Notebook",
            client.repository.ExperimentMetaNames.AUTHOR_NAME: "Watson Studio",
            client.repository.ExperimentMetaNames.DESCRIPTION: "Fashion MNIST HPO - Single Traning Run, 10 HPO iterations",
            client.repository.ExperimentMetaNames.EVALUATION_METHOD: "multiclass",
            client.repository.ExperimentMetaNames.EVALUATION_METRICS: ["val_acc"],
            client.repository.ExperimentMetaNames.TRAINING_DATA_REFERENCE: TRAINING_DATA_REFERENCE,
            client.repository.ExperimentMetaNames.TRAINING_RESULTS_REFERENCE: TRAINING_RESULTS_REFERENCE,
            client.repository.ExperimentMetaNames.TRAINING_REFERENCES: [
                        {
                            "name": "HPO-JOB-21EPOCHS-FMNIST",
                            "training_definition_url": definition_1_url,
                            "compute_configuration": {"name": "k80x2"},
                            "hyper_parameters_optimization": HPO
                            
                        }
                    ]
}

#### Store experiment in Watson Machine Learning repository

In [ ]:
# get the details
experiment_details = client.repository.store_experiment(meta_props=experiment_metadata)

experiment_uid = client.repository.get_experiment_uid(experiment_details)
print(experiment_uid)

#### LIST stored experiments

In [ ]:
# dump the experiments w/ metadata to stdout 
client.repository.list_experiments()

**Hint:** You can update experiment definition by calling below method.

In [ ]:
# this cell is not runnable 
updated_experiment_details = client.repository.update_experiment(experiment_uid, experiment_metadata)

#### GET experiment definition details

In [ ]:
#details is a python dict
details = client.repository.get_experiment_details(experiment_uid)

In [ ]:
details

#### Delete experiment definition from repository

<a id="run"></a>
## 4. Run experiment

### 4.1 Running experiments

In [ ]:
# run the experiment! 
experiment_run_details = client.experiments.run(experiment_uid, asynchronous=True)

As we can see experiment run has been triggered.

#### LIST experiment runs
You can list experiment run using `client.experiments.list_runs()` method.

In [ ]:
client.experiments.list_runs()

#### GET experiment run UID

In [ ]:
experiment_run_uid = client.experiments.get_run_uid(experiment_run_details)
print(experiment_run_uid)

#### LIST training runs triggered by experiment run

**Hint:** Please run the cell below several times during the run to see updates. Or, you can monitor for a few minutes. 

In [ ]:
# query the service ~ every minute
import time 

f = lambda x: time.sleep(6) if x%10!=0 else client.experiments.list_training_runs(experiment_run_uid)
[f(i) for i in range (60)]

#### GET run details
Use below method to get particular experiment run details.

In [ ]:
experiment_run_details = client.experiments.get_run_details(experiment_run_uid)

In [ ]:
experiment_run_details

#### GET experiment run status
You can check experiment run status by calling `client.experiments.get_status(run_uid)` method. This is quite useful when running experiment in background.

In [ ]:
client.experiments.get_status(experiment_run_uid)

#### GET experiment details
Use below method to get all information about particular experiment.

In [ ]:
experiment_details = client.experiments.get_details(experiment_uid)

You can use below print method to display experiment details (change cell type to code).

In [ ]:
print(json.dumps(experiment_details, indent=2))

#### GET training runs uids

In [ ]:
experiment_run_details = client.experiments.get_run_details(experiment_run_uid)
training_run_uids = client.experiments.get_training_uids(experiment_run_details)

for i in training_run_uids:
    print(i)

#### Delete experiment-run

You can delete experiment runs and results by running below method.

**Tip:** the `delete` method can be also used to terminate experiment run

### 4.2 Monitoring experiment

#### MONITOR experiment run
You can monitor experiment run by calling `client.experiments.monitor_logs(run_uid)`. This method will stream training logs content to console.

**Tip:** You can also monitor particular training run by calling `client.training.monitor_logs(training_run_uid)`.
To get training_run_uid you can call method `client.experiments.list_training_runs(experiment_run_uid)`

In [ ]:
client.experiments.monitor_logs(experiment_run_uid)

**Note:** Before going to next section make sure that your experiment run has completed.

In [ ]:
client.experiments.get_status(experiment_run_uid)['state']

### 4.3 Quality metrics

#### Accessing metrics data

You can get final evaluation metrics by running below cell.

In [ ]:
metrics = client.experiments.get_latest_metrics(experiment_run_uid)

You can get all evaluation metrics by running below code.

In [ ]:
all_metrics = client.experiments.get_metrics(experiment_run_uid)

In [ ]:
all_metrics

#### Visualize training results using plotly.

In [ ]:
!pip install --quiet cufflinks

In [ ]:
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go

init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

Let's put our final metrics data into pandas dataframe.

In [ ]:
import pandas as pd

metrics_df = pd.DataFrame(columns=['GUID', 'NAME', 'METRIC NAME', 'METRIC VALUE'])

for m in metrics:
    for v in m['metrics']['values']:
            metrics_df = metrics_df.append({'GUID': m['training_guid'], 'NAME': m['training_reference_name'], 'METRIC NAME': v['name'], 'METRIC VALUE': v['value']}, ignore_index=True)
    
metrics_df

Visualize metrics data using grouped bar chart.

In [ ]:
data = []

for i in list(pd.unique(metrics_df['METRIC NAME'])):
    data.append(go.Bar(x=metrics_df[metrics_df['METRIC NAME'].isin([i])]['GUID'] + ' (' + metrics_df[metrics_df['METRIC NAME'].isin([i])]['NAME'] + ')', y=metrics_df[metrics_df['METRIC NAME'].isin([i])]['METRIC VALUE'], name=i))


layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

## 5. Create online deployment

You can deployed stored model as webservice (online) by running below method.

### 5.1 Store trained model

#### Call this method to save model in Watson Machine Learning repository

In [ ]:
# get the best model
best_model_uid = metrics_df.sort_values(['METRIC NAME', 'METRIC VALUE'], ascending=False).iloc[0,0]
metrics_df.sort_values(['METRIC NAME', 'METRIC VALUE'], ascending=False)

In [ ]:
best_model_uid

In [ ]:
saved_model_details = client.repository.store_model(best_model_uid, {'name': 'MNIST best model'})

**Tip:** Run below code to extract stored model guid from dictionary.

In [ ]:
model_guid = client.repository.get_model_uid(saved_model_details)
print("Saved model guid: " + model_guid)

#### Call this method to list stored models

In [ ]:
client.repository.list_models()

### 5.2 Create online deployment

In [ ]:
deployment_details = client.deployments.create(name="Fashion-MNIST-Keras-Dep", model_uid=model_guid)

Extract `scoring endpoint` from deployment details.

In [ ]:
scoring_url = client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

## 6. Scoring

Prepare sample scoring data to score deployed model.

In [ ]:
from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

Let's plot two items.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
for i, image in enumerate([x_test[9], x_test[19]]):
    plt.subplot(2, 2, i + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')

Our input node expects to get data with shape (28,28,1) so we need to reshape our two digits.

In [ ]:
image_1 = np.expand_dims(x_test[0], axis=2)
image_2 = np.expand_dims(x_test[1], axis=2)

Prepare scoring payload and score.

In [ ]:
scoring_data = {'values': [image_1.tolist(), image_2.tolist()]}

In [ ]:
predictions = client.deployments.score(scoring_url, scoring_data)
print("Scoring result: " + str(predictions))

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use `watson-machine-learning-client` to run experiments. Check out our [Online Documentation](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

**Adam Massachi** is a Data Scientist with the Watson Studio offering management team at IBM. 

Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.